In [1]:
from main import *
import json
import numpy as np
from tqdm import tqdm
import pandas as pd
import time
import os

data = json.loads(open("PGNs/data.json", "r").read())
cohere_llm, openai_llm, gemini_llm = init_llm()

np.random.seed(42)
RAND_COUNT = 25
steps = np.random.randint(5,21, size=(RAND_COUNT)).tolist()  # 5-20
total_games = np.random.randint(1,21, size=(RAND_COUNT)).tolist()  # 1-20, then 50-100
num = np.random.randint(1,6, size=(RAND_COUNT)).tolist()  # 1-5

In [2]:
# Example key-val pair
# timeTaken: {(steps, total_games, num): {output_str: "", avg_score: 0, time_taken: 0}}
param_results = {}

# load from file
if os.path.exists("param_results.json"):
    with open("param_results.json", "r") as f:
        param_results = json.load(f)

for i in tqdm(range(RAND_COUNT)):
    # Grabbing random param values
    steps_i = steps[i]
    total_games_i = total_games[i]
    num_i = num[i]

    # Skip if already in param_results
    if str((steps_i, total_games_i, num_i)) in param_results:
        print(f"{i + 1}. Skipping {str((steps_i, total_games_i, num_i))}")
        continue

    # Run with param values
    start_time = time.time()
    output_str, avg_score = run(
        steps=steps_i,
        total_games=total_games_i,
        num=num_i,
        llm=gemini_llm,
        data=data,
    )
    end_time = time.time()
    time_taken = end_time - start_time

    # Save to param_results
    param_results[str((steps_i, total_games_i, num_i))] = {
        "output_str": output_str,
        "avg_score": avg_score,
        "time_taken": time_taken
    }

    # Save to file
    with open("param_results.json", "w") as f:
        json.dump(param_results, f)

100%|██████████| 25/25 [00:00<00:00, 111669.44it/s]

1. Skipping (11, 6, 4)
2. Skipping (8, 2, 4)
3. Skipping (17, 1, 4)
4. Skipping (19, 12, 4)
5. Skipping (15, 12, 5)
6. Skipping (12, 17, 3)
7. Skipping (17, 10, 1)
8. Skipping (9, 16, 4)
9. Skipping (11, 15, 2)
10. Skipping (14, 15, 4)
11. Skipping (7, 19, 2)
12. Skipping (11, 12, 2)
13. Skipping (15, 20, 4)
14. Skipping (15, 3, 5)
15. Skipping (12, 5, 2)
16. Skipping (9, 19, 2)
17. Skipping (8, 7, 4)
18. Skipping (12, 9, 2)
19. Skipping (12, 7, 2)
20. Skipping (7, 18, 4)
21. Skipping (10, 4, 4)
22. Skipping (9, 14, 1)
23. Skipping (6, 18, 5)
24. Skipping (12, 9, 5)
25. Skipping (16, 2, 2)


### Plotting Results

In [5]:
# df to display results using pairplot
df = pd.DataFrame.from_dict(param_results, orient="index")
df["steps"] = df.index.map(lambda x: eval(x)[0])
df["total_games"] = df.index.map(lambda x: eval(x)[1])
df["num"] = df.index.map(lambda x: eval(x)[2])
df

import plotly.express as px

# Assuming `df` is a DataFrame containing your experiment results
# with columns for the three hyperparameters and a column for the performance metric

fig = px.parallel_coordinates(df, color="avg_score", dimensions=["steps", "total_games", "num", "avg_score", "time_taken"],
                              labels={
                                  "steps": "Steps",
                                  "totoal_games": "Total Games",
                                  "num": "Num",
                                  "avg_score": "Average Score",
                                  "time_taken": "Time Taken"
                              })

fig.show()


In [13]:
# Scatter Plot of Steps vs. Average Score
plot = px.scatter(df, x="num", y="avg_score", color="time_taken")
plot.show()

In [15]:
import plotly.graph_objects as go
fig = go.Figure(data=[go.Scatter3d(x=df["steps"], y=df["total_games"], z=df["avg_score"], mode='markers', marker=dict(size=12, color=df["avg_score"], colorscale='Viridis'))])
fig.show()